### Notebook to process benchmar results

Please run this notebook after running all the benchmarks and storing them in the `results` dir. This will export them in the desired format for the single node benchmark plots of [qdrant.tech/benchmarks](https://qdrant.tech/benchmarks)

In [1]:
from pathlib import Path
import re
import json
import pandas as pd
from datetime import datetime, timezone

In [2]:
DATA_DIR = Path().resolve().parent / "results"
DATA_DIR, list(DATA_DIR.glob("*.json"))[0].name

(PosixPath('/Users/dvir/Code/vector-db-benchmark/results'),
 'vectorsets-q8-m-64-ef-512-random-100-search-2-2025-03-18-23-31-46.json')

In [3]:
PATH_REGEX = re.compile(r"(?P<engine_name>("
                        r"?P<engine>[a-z\-]+)"
                        r"\-(?P<quant>[a-zA-Z0-9\-]+)"
                        r"\-m\-(?P<m>[0-9]+)"
                        r"\-ef\-(?P<ef>[0-9]+)"
                        r")"
                        r"\-(?P<dataset>[a-zA-Z0-9\-]+)"
                        r"\-(?P<operation>(search)|(upload))"
                        r"(\-(?P<search_index>[0-9]{1,2})\-)?"
                        r"\-?(?P<date>.*)\.json")

In [ ]:
upload_results, search_results = [], []

for path in DATA_DIR.glob("*.json"):
    match = PATH_REGEX.match(path.name)
    if match is None:
        continue
        
    experiment = match.groupdict()
    
    with open(path, "r") as fp:
        stats = json.load(fp)

    entry = [match["engine"], match["m"], match["ef"], match["quant"],
             match["dataset"], match["search_index"], match["date"], 
             stats["params"], stats["results"]]
    if experiment["operation"] == "search":
        search_results.append(entry)
    elif experiment["operation"] == "upload":
        upload_results.append(entry)

len(upload_results), len(search_results)

[['vectorsets',
  '64',
  '512',
  'q8',
  'random-100',
  '2',
  '2025-03-18-23-31-46',
  {'dataset': 'random-100',
   'experiment': 'vectorsets-q8-m-64-ef-512',
   'engine': 'vectorsets',
   'parallel': 1,
   'search_params': {'ef': 256}},
  {'total_time': 0.0025846249773167074,
   'mean_time': 0.00020889558945782482,
   'mean_precisions': 1.0,
   'std_time': 0.00012517310030909733,
   'min_time': 0.00012079102452844381,
   'max_time': 0.0005068749887868762,
   'rps': 3869.033259278392,
   'p95_time': 0.0004580310720484703,
   'p99_time': 0.000497106205439195}],
 ['vectorsets',
  '64',
  '512',
  'q8',
  'random-100',
  '5',
  '2025-03-18-23-32-25',
  {'dataset': 'random-100',
   'experiment': 'vectorsets-q8-m-64-ef-512',
   'engine': 'vectorsets',
   'parallel': 100,
   'search_params': {'ef': 128}},
  {'total_time': 4.754267499956768,
   'mean_time': 0.0019451417960226537,
   'mean_precisions': 1.0,
   'std_time': 0.0037093798096747236,
   'min_time': 0.0004131249734200537,
   'max

In [5]:
column_names = ["engine", "m", "ef", "quant", "dataset", "search_index", "date", "params", "results"]

In [6]:
upload_df = pd.DataFrame(upload_results, columns=column_names) \
    .drop(columns="search_index")
upload_df["date"] = pd.to_datetime(upload_df["date"], format="%Y-%m-%d-%H-%M-%S")
upload_df = upload_df.sort_values("date", ascending=False) \
    .groupby(["engine", "m", "ef", "quant", "dataset"]) \
    .last()
upload_df = pd.concat([upload_df, upload_df["results"].apply(pd.Series)], axis=1)
upload_df = upload_df.drop(columns="results")

print(len(upload_df))

upload_df.sort_values("total_time", ascending=True).head(n=5)

1


,,,,,date,params,post_upload,upload_time,total_time
engine,m,ef,quant,dataset,,,,,
vectorsets,64,512,q8,random-100,2025-03-18 23:31:46,"{'experiment': 'vectorsets-q8-m-64-ef-512', 'e...",{},3.590291,3.590334


In [ ]:
search_df = pd.DataFrame(search_results, columns=column_names)
search_df["date"] = pd.to_datetime(search_df["date"], format="%Y-%m-%d-%H-%M-%S")
search_df = search_df.sort_values("date", ascending=False) \
    .groupby(["engine", "m", "ef", "dataset", "quant", "search_index"]) \
    .first()

print(len(search_df))

for column_name in ["params", "results"]:
    search_df = pd.concat([search_df, search_df[column_name].apply(pd.Series)], axis=1)
    search_df = search_df.drop(columns=column_name)
search_df.sort_values("rps", ascending=False).head(n=10)

8


date  \
engine     m  ef  dataset    quant search_index                       
vectorsets 64 512 random-100 q8    3            2025-03-18 23:31:46   
                                   1            2025-03-18 23:31:46   
                                   2            2025-03-18 23:31:46   
                                   0            2025-03-18 23:31:46   
                                   4            2025-03-18 23:32:05   
                                   6            2025-03-18 23:32:44   
                                   7            2025-03-18 23:33:03   
                                   5            2025-03-18 23:32:25   

                                                    dataset  \
engine     m  ef  dataset    quant search_index               
vectorsets 64 512 random-100 q8    3             random-100   
                                   1             random-100   
                                   2             random-100   
                                   0             random-100   
                                   4             random-100   
                                   6             random-100   
                                   7             random-100   
                                   5             random-100   

                                                                experiment  \
engine     m  ef  dataset    quant search_index                              
vectorsets 64 512 random-100 q8    3             vectorsets-q8-m-64-ef-512   
                                   1             vectorsets-q8-m-64-ef-512   
                                   2             vectorsets-q8-m-64-ef-512   
                                   0             vectorsets-q8-m-64-ef-512   
                                   4             vectorsets-q8-m-64-ef-512   
                                   6             vectorsets-q8-m-64-ef-512   
                                   7             vectorsets-q8-m-64-ef-512   
                                   5             vectorsets-q8-m-64-ef-512   

                                                     engine  parallel  \
engine     m  ef  dataset    quant search_index                         
vectorsets 64 512 random-100 q8    3             vectorsets         1   
                                   1             vectorsets         1   
                                   2             vectorsets         1   
                                   0             vectorsets         1   
                                   4             vectorsets       100   
                                   6             vectorsets       100   
                                   7             vectorsets       100   
                                   5             vectorsets       100   

                                                search_params  total_time  \
engine     m  ef  dataset    quant search_index                             
vectorsets 64 512 random-100 q8    3              {'ef': 512}    0.002255   
                                   1              {'ef': 128}    0.002358   
                                   2              {'ef': 256}    0.002585   
                                   0               {'ef': 64}    0.003914   
                                   4               {'ef': 64}    2.029471   
                                   6              {'ef': 256}    2.790438   
                                   7              {'ef': 512}    3.443149   
                                   5              {'ef': 128}    4.754267   

                                                 mean_time  mean_precisions  \
engine     m  ef  dataset    quant search_index                               
vectorsets 64 512 random-100 q8    3              0.000183              1.0   
                                   1              0.000176              1.0   
                                   2              0.000209              1.0   
                                   0       

In [12]:
# # Option 1: Check what's in your index and columns
# print("search_df index name:", search_df.index.name or search_df.index.names)
# print("search_df columns:", search_df.columns.tolist())
# print("upload_df index name:", upload_df.index.name or upload_df.index.names)
# print("upload_df columns:", upload_df.columns.tolist())

# # Option 2: Reset index but specify a different name for the index column
# _search = search_df.reset_index()
# _upload = upload_df.reset_index()
# print("search_df index name:", _search.index.name or _search.index.names)
# print("search_df columns:", _search.columns.tolist())

# print("_upload index name:", _upload.index.name or _upload.index.names)
# print("_upload columns:", _upload.columns.tolist())

# joined_df = _search.merge(_upload, how="left", on=["engine", "m", "ef", "quant", "dataset"], suffixes=("_search", "_upload"))
# print(len(joined_df))
# joined_df

# Reset the indices of both dataframes to make columns
search_reset = search_df.reset_index()
upload_reset = upload_df.reset_index()

# Join on common columns
joined_df = search_reset.merge(upload_reset, 
                             how="left", 
                             on=["engine", "m", "ef", "quant", "dataset"],
                             suffixes=("", "_upload"))

# Rename any conflicting columns to match what's expected in cell 10
joined_df = joined_df.rename(columns={
    "total_time": "total_time_search",
    "total_time_upload": "total_time_upload"
})

print(f"Joined dataframe has {len(joined_df)} rows")
joined_df.head(2)

ValueError: cannot insert dataset, already exists

In [ ]:
json_all = []
json_1_or_100_thread = []

for index, row in joined_df.reset_index().iterrows():
    engine_params = {}
    if isinstance(row['search_params'], dict):
        engine_params.update(row['search_params'])
    if isinstance(row['params'], dict):
        engine_params.update(row['params'])

    engine_name = row['engine']

    if engine_name == "qdrant-rps" or engine_name == "qdrant-bq-rps" or engine_name == "qdrant-sq-rps":
        engine_name = "qdrant"

    json_object = {
        "engine_name": engine_name,
        "setup_name": f"{row['engine']}-m-{row['m']}-ef-{row['ef']}",
        "dataset_name": row['dataset'],
        # "search_idx": row['search_index'],
        "upload_time": row['upload_time'],
        "total_upload_time": row['total_time_upload'],
        "p95_time": row['p95_time'],
        "rps": row['rps'],
        "parallel": row['parallel'],
        "p99_time": row['p99_time'],
        "mean_time": row['mean_time'],
        "mean_precisions": row['mean_precisions'],
        "engine_params": engine_params,
    }
    json_all.append(json_object)
    
    parallel = row['parallel']

    if parallel == 1 or parallel == 100:
        json_1_or_100_thread.append(json_object)

format = '%Y-%M-%d' # T%H:%M:%S
now = datetime.now().replace(tzinfo=timezone.utc).strftime(format)

Path(f"results-{now}.json").write_text(json.dumps(json_all, indent=2))
Path(f"results-1-100-threads-{now}.json").write_text(json.dumps(json_1_or_100_thread, indent=2))

json_1_or_100_thread[-1], len(json_all), len(json_1_or_100_thread)

KeyError: 'upload_time'